<img align='center' style='max-width: 1000px' src='banner.png'>

<img align="right" style='max-width: 200px; height: auto' src='hsg_logo.png'>

##  Lab 07 - Recurrent Neural Networks (RNNs)

GSERM Summer School 2024, Deep Learning: Fundamentals and Applications, University of St. Gallen

The lab environment is based on Jupyter Notebooks (https://jupyter.org), which provide an interactive platform for performing a variety of statistical evaluations and data analyses. In this lab, we will learn how to apply another type of deep learning technique referred to as **Long Short-Term Memory (LSTM)** neural networks. Unlike standard feedforward neural networks, LSTMs encompass feedback connections that make them suitable for processing not only single data points (such as images) but also entire sequences of data, such as speech, video, or financial time series.

LSTMs have a rich history with pivotal contributions from researchers like *Sepp Hochreiter* and *Jürgen Schmidhuber*, who introduced these networks to address the vanishing gradient problem in training recurrent neural networks (RNNs). LSTMs have since become a cornerstone in the field of sequence prediction, significantly advancing the capabilities of time-series forecasting and natural language processing.

In this lab, we will use the `PyTorch` library to implement and train an **LSTM-based neural network**. The network will be trained on the historical daily (in-sample) returns of an exemplary financial stock. Once the network is trained, we will use the learned model to predict future (out-of-sample) returns. Finally, we will convert the predictions into tradeable signals and backtest these signals accordingly.

The figure below illustrates a high-level view of the machine learning process we aim to establish in this lab.

<img align='center' style='max-width: 1000px' src='splash.png'>

As always, pls. don't hesitate to ask all your questions either during the lab, post them in our CANVAS (StudyNet) forum (https://learning.unisg.ch), or send us an email (using the course email).

## 1. Lab Objectives:

After today's lab, you should be able to:

> 1. **Understand Long Short-Term Memory (LSTM) Neural Network Design:** Learn the fundamental concepts and architectural design of LSTMs.
> 2. **Implement and Train an LSTM Model:** Gain hands-on experience with PyTorch to implement, train, and evaluate LSTM models.
> 3. **Apply LSTM Models for Time-Series Prediction:** Use LSTMs to predict future data points in financial time series datasets.
> 4. **Evaluate and Interpret Model Performance:** Evaluate the LSTM model's performance using relevant metrics and interpret the prediction results.
> 5. **Visualize and Interpret Time-Series Data:** Visualize and interpret the model's predictions to gain deeper insights into temporal patterns in the data.

Before we start let's watch a motivational video:

In [ ]:
from IPython.display import YouTubeVideo
# "AlphaStar: The inside story"
# YouTubeVideo('UuhECwm31dM', width=800, height=400)

## 2. Setup of the Jupyter Notebook Environment

Similar to the previous labs, we need to import several Python libraries that facilitate data analysis and visualization. We will primarily use `PyTorch`, `NumPy`, `Pandas`, `Scikit-learn`, `Matplotlib`, `Seaborn`, and a few utility libraries throughout this lab:

In [ ]:
# import python data science and utility libraries
import os, io, urllib, itertools
import datetime as dt
import pandas as pd
import pandas_datareader as dr
import numpy as np

Install the Python `bt` backtesting library and upgrade the `Pandas` datareader (a restart of the Colab runtime environment is potentially required):

In [ ]:
!pip install bt
!pip install --upgrade pandas-datareader

Import the `bt backtesting` library:

In [ ]:
import bt as bt # library to backtest trading signals

Import the `yahoo! finance` data retrieval library:

In [ ]:
import yfinance as yf

Import `Python` machine learning and deep learning libraries:

In [ ]:
# pytorch libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils import data
from torch.utils.data import dataloader

Import `Matplotlib` and `Seaborn` data visualization libraries:

In [ ]:
# visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

# set seaborn theme
sns.set_theme()

# set general plotting parameters
plt.rcParams['figure.figsize'] = [10, 5]
plt.rcParams['figure.dpi']= 150

Turn off possible warnings, e.g., due to future changes in the libraries:

In [ ]:
import warnings
warnings.filterwarnings('ignore')

Enable inline plotting with `Matplotlib`:

In [ ]:
%matplotlib inline

Import Google's `GDrive` connector and mount your `GDrive` directories:

In [ ]:
# import the Google Colab GDrive connector
from google.colab import drive

# mount GDrive inside the Colab notebook
drive.mount('/content/drive')

Create a structure of Colab Notebook sub-directories inside of GDrive to store (1) the `GDrive` notebooks in general, (2) saving the original data, and (3) the trained models:

In [ ]:
# create Colab Notebooks directory
notebook_directory = '/content/drive/MyDrive/Colab Notebooks'
if not os.path.exists(notebook_directory): os.makedirs(notebook_directory)

 # create data sub-directory inside the Colab Notebooks directory
data_directory = '/content/drive/MyDrive/Colab Notebooks/data_lstm'
if not os.path.exists(data_directory): os.makedirs(data_directory)

 # create models sub-directory inside the Colab Notebooks directory
models_directory = '/content/drive/MyDrive/Colab Notebooks/models_lstm'
if not os.path.exists(models_directory): os.makedirs(models_directory)

Set a random `seed` value to obtain reproducible results:

In [ ]:
# init deterministic seed
seed_value = 1234
np.random.seed(seed_value) # set numpy seed
torch.manual_seed(seed_value); # set pytorch seed CPU

Google Colab provides free GPUs for running notebooks. However, if you execute this notebook as is, it will use your device's CPU. To run the lab on a GPU, go to `Runtime` > `Change runtime type` and set the Runtime type to `GPU` in the drop-down menu. Running this lab on a CPU is fine, but you will find that GPU computing is faster. *CUDA* indicates that the lab is being run on a GPU.

Enable GPU computing by setting the device flag and initializing a CUDA s

In [ ]:
# set cpu or gpu enabled device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu').type

# init deterministic GPU seed
torch.cuda.manual_seed(seed_value)

# log type of device enabled
now = dt.datetime.utcnow().strftime("%Y%m%d-%H:%M:%S")
print('[LOG {}] notebook with \'{}\' computation enabled'.format(str(now), str(device)))

## 3. Dataset Download and Data Assessment

In this section of the lab notebook, we will download and access historical daily stock market data for the **"International Business Machines" (IBM)** corporation (ticker symbol: "IBM") from **01/01/2000** to **31/12/2017**. We will use the `datareader` from the `Pandas` library to interface with the Yahoo Finance API.

To start the data download, let's specify the start and end dates for the stock market data:

In [ ]:
start_date = dt.datetime(2000, 1, 1)
end_date = dt.datetime(2017, 12, 31)

Download the daily stock market data for *International Business Machines (IBM)*:

In [ ]:
stock_data = yf.download('IBM', start=start_date, end=end_date)

Inspect the top 10 records of the retrieved IBM stock market data:

In [ ]:
stock_data.head(10)

Next, evaluate the data quality by generating summary statistics for the retrieved data:

In [ ]:
stock_data.describe()

Visualize the daily closing prices of the "International Business Machines" (IBM) stock market data:

In [ ]:
# define plot size 
plt.rcParams['figure.figsize'] = [15, 5]

# initialize plot
fig = plt.figure()
ax = fig.add_subplot(111)

# plot reconstruction error scatter plot
ax.plot(stock_data.index, stock_data['Close'], color='green')

for tick in ax.get_xticklabels():
    tick.set_rotation(45)

# set x-axis labels and limits
ax.set_xlabel('[time]', fontsize=10)
ax.set_xlim([pd.to_datetime('01-01-2000'), pd.to_datetime('31-12-2017')])

# set y-axis labels and limits
ax.set_ylabel('[stock adj. closing price]', fontsize=10)
ax.set_ylim(20, 220)

# set plot title
plt.title('International Business Machines (IBM) - Daily Historical Stock Closing Prices', fontsize=10);

Save the downloaded and validated stock market data to the local data directory:

In [ ]:
# save retrieved data to local data directory
stock_data.to_csv(os.path.join(data_directory, 'ibm_data_2010_2017_daily.csv'), sep=';', encoding='utf-8')

## 4. Dataset Pre-Processing

In this section, we will calculate the daily returns from the retrieved daily closing prices. We will then convert the time series of daily returns into a set of sequences $s$ of $n$ time steps each. These sequences will be used to train a model using a Long Short-Term Memory (LSTM) neural network.

### 4.1 Weekend and Holiday Padding

We will forward propagate the last valid price observation to the next available valid price using the Pandas `reindex()` function. This ensures we obtain market price information for weekends and holidays:

In [ ]:
# fill weekends and holidays
stock_data = stock_data.reindex(index=pd.date_range(stock_data.index.min(), stock_data.index.max()), method='ffill')

Inspect the padded stock market data for "International Business Machines" (IBM):

In [ ]:
stock_data.head(10)

Check the number of records obtained after padding the data:

In [ ]:
stock_data.shape

### 4.2 Daily Return Calculation

Calculate the daily returns of the "International Business Machines" (IBM) closing prices using the Pandas `pct_change()` function:

In [ ]:
stock_data['RETURN'] = stock_data['Close'].pct_change()

Inspect the calculated daily returns of the closing prices:

In [ ]:
stock_data['RETURN']

Visually inspect the calculated daily returns:

In [ ]:
plt.rcParams['figure.figsize'] = [15, 5]
fig = plt.figure()
ax = fig.add_subplot(111)

# plot reconstruction error scatter plot
ax.plot(stock_data.index, stock_data['RETURN'], color='green')

for tick in ax.get_xticklabels():
    tick.set_rotation(45)

# set axis labels and limits
ax.set_xlabel('[time]', fontsize=10)
ax.set_xlim([pd.to_datetime('01-01-2000'), pd.to_datetime('31-12-2017')])
ax.set_ylabel('[daily stock returns]', fontsize=10)

# set plot title
plt.title('International Business Machines (IBM) - Daily Historical Stock Closing Prices', fontsize=10);

### 4.3 Conduct Train-Test Split

To understand and evaluate the performance of any trained **supervised machine learning** model, it is good practice to divide the dataset into a **training set** or **"in-sample"** data (used solely for model training) and a **test set** or **"out-of-sample"** data (used solely for model testing).

<img align='center' style='max-width: 500px' src='traintestsplit.png'>

We set the split fraction for training sequences to **90%** of the total number of sequences obtained:

In [ ]:
split_fraction = 0.9
split_row = int(stock_data.shape[0] * split_fraction)

Split the obtained returns into training ("in-sample") returns $r^{i}_{train}$ and validation ("out-of-sample") returns $r^{i}_{valid}$:

In [ ]:
train_stock_data = stock_data.iloc[:split_row]
valid_stock_data = stock_data.iloc[split_row:]

Visually inspect the obtained training and validation stock returns:

In [ ]:
plt.rcParams['figure.figsize'] = [15, 5]
fig = plt.figure()
ax = fig.add_subplot(111)

# plot daily stock returns
ax.plot(stock_data.index[:split_row,], train_stock_data['RETURN'], c='green', label='train (green)')
ax.plot(stock_data.index[split_row:,], valid_stock_data['RETURN'], c='grey', label='valid (grey)')

# rotate x-labels 45 degree angle
for tick in ax.get_xticklabels():
    tick.set_rotation(45)

# set axis labels and limits
ax.set_xlabel('[time]', fontsize=10)
ax.set_xlim([pd.to_datetime('01-01-2000'), pd.to_datetime('31-12-2017')])
ax.set_ylabel('[daily stock returns]', fontsize=10)

# set plot legend
plt.legend(loc="lower right", numpoints=1, fancybox=True)

# set plot title
plt.title('International Business Machines (IBM) - Daily Historical Stock Returns', fontsize=10);

Determine count (shape) of daily return train sequences $r^{i}_{train}$:

In [ ]:
train_stock_data.shape

Determine count (shape) of daily return train sequences $r^{i}_{valid}$:

In [ ]:
valid_stock_data.shape

### 4.4 Transform into Sequences

In the following, we determine the number of return time-steps $n$ each individual sequence $s^{i}$ should be comprised of. Each sequence is thereby determined by the number of predictor (return) time-steps $t$ and the prediction (return) horizon $h = t+1$.

<img align='center' style='max-width: 500px' src='timesteps.png'>

In this example, we will set the number of predictor (return) time-steps to $t$=4. This means that the input sequence of each sample is a vector of 4 sequential daily stock returns (please note, the choice of $t$=4 is arbitrary and should be selected through experimentation). Furthermore, we set the predicted return horizon to 1, which specifies that we aim to forecast a single future time-step.

In [ ]:
time_steps = 4 # number of predictor timesteps
horizon = 1 # number of timesteps to be predicted
sequence_length = time_steps + horizon # determine sequence length

Next, we extract sequences $s^i$ of 5 time steps.

We will use a "rolling window" approach to iterate over the entire sequence of daily stock returns $r_i$. In each iteration, we extract an individual sequence of stock returns consisting of $n$ time steps. The extracted sequences are then collected into a single DataFrame.

<img align='center' style='max-width: 900px' src='sequences.png'>

Determine the maximum number of training ("in-sample") sequences:

In [ ]:
# determine max train index
max_train_index = ((train_stock_data.shape[0] // sequence_length) - 1) * sequence_length

Extract individual training sequences of length $5$ from the calculated daily returns:

In [ ]:
# initialize lists to collect sequences
train_stock_sequence_data_date = []
train_stock_sequence_data = []

# iterate over the distinct daily returns of the training dataset
for i in range(1, max_train_index):
    
    # determine sequence of timesteps and daily returns
    sequence_dates = train_stock_data.index[i:i + sequence_length].to_numpy()
    sequence_returns = train_stock_data['RETURN'][i:i + sequence_length].to_numpy()

    # collect sequence of timesteps and daily returns
    train_stock_sequence_data_date.append(sequence_dates)
    train_stock_sequence_data.append(sequence_returns)

# convert lists to numpy arrays
train_stock_sequence_data_date = np.array(train_stock_sequence_data_date)
train_stock_sequence_data = np.array(train_stock_sequence_data)

Determine the total number of extracted training sequences:

In [ ]:
train_stock_sequence_data.shape

Inspect the top five extracted sequences of training time steps:

In [ ]:
train_stock_sequence_data_date[0:5,]

Inspect the top five extracted sequences of training returns $s^{i}_{train}=\\{r_{t-n-1}, ..., r_{t-1}, r_{t}\\}$:

In [ ]:
train_stock_sequence_data[0:5,]

Determine the maximum number of validation ("out-of-sample") sequences:

In [ ]:
# determine max valid index
max_valid_index = ((valid_stock_data.shape[0] // sequence_length) - 1) * sequence_length

Extract individual validation sequences of length $5$ from the calculated daily returns:

In [ ]:
# initialize lists to collect sequences
valid_stock_sequence_data_date = []
valid_stock_sequence_data = []

# iterate over the distinct daily returns of the validation dataset
for i in range(1, max_valid_index):
    
    # determine sequence of timesteps and daily returns
    sequence_dates = valid_stock_data.index[i:i + sequence_length].to_numpy()
    sequence_returns = valid_stock_data['RETURN'][i:i + sequence_length].to_numpy()

    # collect sequence of timesteps and daily returns
    valid_stock_sequence_data_date.append(sequence_dates)
    valid_stock_sequence_data.append(sequence_returns)

# convert lists to numpy arrays
valid_stock_sequence_data_date = np.array(valid_stock_sequence_data_date)
valid_stock_sequence_data = np.array(valid_stock_sequence_data)

Determine the total number of obtained validation sequences:

Determine the total number of extracted validation sequences:

In [ ]:
valid_stock_sequence_data.shape

Inspect the top five extracted sequences of validation time steps:

In [ ]:
valid_stock_sequence_data_date[0:5,]

Inspect the top five extracted sequences of validation returns $s^{i}_{valid}=\{r_{t-n-1}, ..., r_{t-1}, r_{t}\}$:

In [ ]:
valid_stock_sequence_data[0:5,]

### 4.5 Conduct Input-Target Split

Before we continue the data pre-processing, let's briefly revisit how RNNs, or more specifically, LSTM-based neural networks can be trained to predict the next element of an input sequence. The illustration below is derived from the "Next Word Predictor" example discussed in the course. For each **input return** $r_{i}$ in the input return training sequence $s^i$, the LSTM is supposed to learn to **predict the return** of the next time-step $\hat{r}_{i+1}$. To make such a future return $\hat{r}_{i+1}$ prediction, the LSTM uses its learned hidden state information $h_{i}$ as well as the current return $r_{i}$ as inputs.

For each time-step, the predicted return $\hat{r}_{i+1}$ is then compared to the **target return** $r_{i+1}$. The discrepancy between them is collected as a loss $\mathcal{L}$ for the distinct time steps. The sum of the individual time-step losses is accumulated as the total loss of a sequence $\mathcal{L}_{All}$.

<img align='center' style='max-width: 600px' src='training.png'>

Separate each training sequence $s^{i}$ into time steps of input returns denoted by $s^{i}_{train, input}=\{r_{t-n-1}, ..., r_{t-1}, r_{t}\}$ and the time step of the predicted target return denoted by $s^{i}_{train, target}=r_{t+1}$.

<img align='center' style='max-width: 700px' src='sequencesplit.png'>

Additionally, we convert both the input returns and the target returns to PyTorch tensors:

In [ ]:
train_sequences_input = torch.from_numpy(train_stock_sequence_data[:, :-1]).float()
train_sequences_target = torch.from_numpy(train_stock_sequence_data[:, 1:]).float()

Separate each validation sequence $s^{i}$ into time steps of input returns denoted by $s^{i}_{valid, input}=\\{r_{t-n-1}, ..., r_{t-1}, r_{t}\\}$ and the time step of the predicted target return denoted by $s^{i}_{valid, target}=r_{t+1}$. Additionally, we convert both the input returns and the target returns to `PyTorch` tensors:

In [ ]:
valid_sequences_input = torch.from_numpy(valid_stock_sequence_data[:, :-1]).float()
valid_sequences_target = torch.from_numpy(valid_stock_sequence_data[:, 1:]).float()

To train an LSTM neural network, we customize the dataset class provided by the PyTorch library. We overwrite the individual functions of the dataset class so that our dataset will supply the neural network with the individual training sequences $s^{i}_{train, input}$ and corresponding targets $s^{i}_{train, target}$ throughout the training process:

In [ ]:
# define daily returns dataset
class DailyReturnsDataset(data.Dataset):

    # define the class constructor
    def __init__(self, sequences, targets):

        # init sequences and corresponding targets
        self.sequences = sequences
        self.targets = targets

    # define the length method 
    def __len__(self):

        # returns the number of samples
        return len(self.targets)

    # define the get item method
    def __getitem__(self, index):

        # determine single sequence and corresponding target
        sequence = self.sequences[index, :]
        target = self.targets[index, :]

        # return sequences and target
        return sequence, target

Once we have specified the daily returns dataset class, we instantiate it using the new daily closing dataset with the prepared training input sequences $s^{i}_{train, input}$ and corresponding targets $s^{i}_{train, target}$:

In [ ]:
train_dataset = DailyReturnsDataset(train_sequences_input, train_sequences_target)

Let's see how it works by getting the 42nd sequence and its corresponding targets:

In [ ]:
train_dataset.__getitem__(42)

## 5. Neural Network Implementation

In this section, we will implement the LSTM architecture for the time series model to be learned. Furthermore, we will specify the loss function, learning rate, and optimization technique used in the network training.

### 5.1. Implementation of the Network Architecture

First, we will implement the architecture of the LSTM neural network used to predict future returns of financial time series data, such as the future returns of a given stock in this example. The neural network, which we name **'LSTMNet'**, consists of three layers in total. The first two layers are LSTM cells, while the third layer is a fully connected linear layer.

<img align='center' style='max-width: 400px' src='lstmnet.png'>

The general LSTM cell structure and the formal definition of its individual gate functions are shown below (not considering the bias of each layer for simplicity):

<img align='center' style='max-width: 700px' src='lstmcell.png'>

(Source: https://pytorch.org/docs/stable/nn.html)

Each LSTM layer consists of an LSTM cell with a **hidden stat**e of **51 dimensions**. The third linear layer compresses the 51 hidden state dimensions of the second LSTM cell into a single output dimension. The single output signal of the linear layer represents the return of the next time step predicted by the neural network. Please note that the choice of the implemented architecture and network hyperparameters is arbitrary and should be thoroughly evaluated and selected through experimentation in a real-world scenario.

In [ ]:
# implement the LSTMNet network architecture
class LSTMNet(nn.Module):

    # define class constructor
    def __init__(self):

        super(LSTMNet, self).__init__()

        # define lstm nn architecture
        self.lstm1 = nn.LSTMCell(1, 51)  # first lstm layer
        self.lstm2 = nn.LSTMCell(51, 51)  # second lstm layer
        self.linear = nn.Linear(51, 1)  # final linear layer

    # define network forward pass
    def forward(self, input):
        
        # init predictions
        predictions = []

        # init the lstm hidden states
        h_t1 = torch.zeros(input.size(0), 51, dtype=torch.float).to(device)
        h_t2 = torch.zeros(input.size(0), 51, dtype=torch.float).to(device)

        # init the lstm cell states
        c_t1 = torch.zeros(input.size(0), 51, dtype=torch.float).to(device)
        c_t2 = torch.zeros(input.size(0), 51, dtype=torch.float).to(device)
        
        # iterate over distinct time steps
        for i, input_t in enumerate(input.chunk(input.size(1), dim=1)):

            # propagate through time step data
            h_t1, c_t1 = self.lstm1(input_t, (h_t1, c_t1))
            h_t2, c_t2 = self.lstm2(h_t1, (h_t2, c_t2))
            prediction = self.linear(h_t2)
            
            # collect predictions
            predictions += [prediction]

        # stack predictions
        predictions = torch.stack(predictions, 1).squeeze(2)

        # return predictions
        return predictions

Now that we have implemented our first LSTM neural network, we are ready to instantiate a model to be trained:

In [ ]:
lstm_model = LSTMNet().to(device)

Once the model is initialized, we can visualize the model structure and review the implemented network architecture by executing the following cell:

In [ ]:
# print the initialized architectures
print('[LOG] LSTMNet architecture:\n\n{}\n'.format(lstm_model))

Finally, let's look at the number of model parameters that we aim to train in the next steps of the notebook:

In [ ]:
# init the number of model parameters
num_params = 0

# iterate over the distinct parameters
for param in lstm_model.parameters():

    # collect number of parameters
    num_params += param.numel()
    
# print the number of model paramters
print('[LOG] Number of to be trained LSTMNet model parameters: {}.'.format(num_params))

Our "simple" `LSTMNet` model already encompasses an impressive **32,284 model parameters** to be trained.

### 5.2. Definition of the Training Loss, Learning Rate, and Optimizer

We are now ready to train the network. However, before starting the training, we need to define an appropriate loss function. Remember, we aim to train our model to learn a set of model parameters $\theta$ that minimize the prediction error between the true return $r_{t+1}$ and the model-predicted return $\hat{r}_{t+1}$ at a given time-step $t+1$ of sequence $s^{i}$. In other words, for a given sequence of historical returns, we aim to learn a function $f_\theta$ that can predict the return of the next time step as accurately as possible, as expressed by:

<center> $\hat{r}_{t+1} = f_\theta(r_{t}, r_{t-1}, ..., r_{t-n})$. </center>

The training objective is to learn a set of optimal model parameters $\theta^*$ that optimize $\min_{\theta} \|r_{t+1} - f_\theta(r_{t}, r_{t-1}, ..., r_{t-n})\|$ over all time steps $t$ contained in the set of training sequences $s_{train}$. To achieve this optimization objective, one typically minimizes a loss function $\mathcal{L_{\theta}}$ while training the neural network. In this lab, we use the **'Mean Squared Error (MSE)'** loss, as denoted by:

<center> $\mathcal{L}^{MSE}_{\theta} (r_{t+1}, \hat{r}_{t+1}) = \frac{1}{N} \sum_{i=1}^N \| r_{t+1} - \hat{r}_{t+1}\|^{2}$, </center>

In [ ]:
loss_function = nn.MSELoss().to(device)

Throughout the training process, the PyTorch library will automatically calculate the loss magnitude, compute the gradient, and update the parameters $\theta$ of the LSTM neural network. We will use the **Adaptive Moment Estimation Optimization" (ADAM)** technique to optimize the network parameters. Furthermore, we specify a constant learning rate of $l = 1 \times 10^{-6}$. For each training step, the optimizer will update the model parameters $\theta$ values according to the degree of prediction error (the MSE loss).

In [ ]:
learning_rate = 1e-06 # set constant learning rate
optimizer = optim.Adam(lstm_model.parameters(), lr=learning_rate) # define optimization technique

Now that we have successfully implemented and defined the three ANN building blocks, let's take some time to review the `LSTMNet` model definition as well as the `MSE loss` function. Please read the above code and comments carefully, and don't hesitate to ask any questions you might have.

## 6. Neural Network Model Training

In this section, we will train the LSTM neural network model (as implemented in the section above) using the prepared dataset of daily return sequences. We will closely examine the distinct training steps and monitor the training progress.

### 6.1. Preparing the Network Training

Let's now start training the model by running the neural network for **5 epochs** in mini-batches of **128 sequences** per batch. This means the entire dataset will be fed to the network **5 times** in chunks of 128 sequences, resulting in **32 mini-batches** (4,068 training sequences / 128 sequences per mini-batch) per epoch:

In [ ]:
# specify the training parameters
num_epochs = 200 # number of training epochs
mini_batch_size = 128 # size of the mini-batches

Furthermore, let's specify and instantiate a corresponding `PyTorch` data loader that feeds the image tensors to our neural network:

In [ ]:
dl = dataloader.DataLoader(train_dataset, batch_size=mini_batch_size, shuffle=True)

### 6.2. Running the Network Training

Finally, we start training the model. The training procedure for each mini-batch is performed as follows: 

>1. Perform a forward pass through the LSTMNet network,
>2. Compute the mean-squared prediction error $\mathcal{L}^{MSE}_{\theta} (r_{t+1}, \hat{r}_{t+1}) = \frac{1}{N} \sum_{i=1}^N \| r_{t+1} - \hat{r}_{t+1}\|^{2}$,
>3. Perform a backward pass through the LSTMNet network, and
>4. Update the parameters of the network $f_\theta(\cdot)$.

To ensure learning while training the LSTM model, we will monitor whether the loss decreases as training progresses. Therefore, we will obtain and evaluate the mean prediction performance over all mini-batches in each training epoch. Based on this evaluation, we can assess the training progress and determine whether the loss is converging (indicating that the model might not improve any further).

The following elements of the network training code below should be given particular attention:
 
>- `loss.backward()` computes the gradients based on the magnitude of the reconstruction loss,
>- `optimizer.step()` updates the network parameters based on the gradients.

In [ ]:
# init collection of training epoch losses
train_epoch_losses = []

# set the model in training mode
lstm_model.train()

# init the best loss
best_loss = np.inf

# iterate over epochs
for epoch in range(0, num_epochs):

    # init collection of mini-batch losses
    train_mini_batch_losses = []
            
    # iterate over mini-batches
    for sequence_batch, target_batch in dl:
        
        # push mini-batch data to computation device
        sequence_batch = sequence_batch.to(device)
        target_batch = target_batch.to(device)

        # predict sequence output
        prediction_batch = lstm_model(sequence_batch)

        # calculate batch loss
        batch_loss = loss_function(prediction_batch, target_batch)

        # run backward gradient calculation
        batch_loss.backward()

        # update network parameters
        optimizer.step()
        
        # collect mini-batch loss
        train_mini_batch_losses.append(batch_loss.data.item())
            
    # determine mean min-batch loss of epoch
    train_epoch_loss = np.mean(train_mini_batch_losses)
    
    # print epoch loss
    now = dt.datetime.utcnow().strftime("%Y%m%d-%H:%M:%S")
    print('[LOG {}] epoch: {} train-loss: {}'.format(str(now), str(epoch), str(train_epoch_loss)))
    
    # determine mean min-batch loss of epoch
    train_epoch_losses.append(train_epoch_loss)
        
    # print epoch and save models
    if epoch % 10 == 0 and epoch > 0:
        
        # case: new best model trained
        if train_epoch_loss < best_loss:
                        
            # store new best model
            model_name = 'best_lstm_model_{}.pth'.format(str(epoch))
            torch.save(lstm_model.state_dict(), os.path.join(models_directory, model_name))
            
            # update best loss
            best_loss = train_epoch_loss
            
            # print epoch loss
            now = dt.datetime.utcnow().strftime("%Y%m%d-%H:%M:%S")
            print('[LOG {}] epoch: {} new best train-loss: {} found'.format(str(now), str(epoch), str(train_epoch_loss)))


Upon successful training, let's visualize and inspect the training loss per epoch:

In [ ]:
# prepare plot
fig = plt.figure()
ax = fig.add_subplot(111)

# add grid
ax.grid(linestyle='dotted')

# plot the training epochs vs. the epochs' prediction error
ax.plot(np.array(range(1, len(train_epoch_losses)+1)), train_epoch_losses, label='epoch loss (blue)')

# add axis legends
ax.set_xlabel("[training epoch $e_i$]", fontsize=10)
ax.set_ylabel("[Prediction Error $\mathcal{L}^{MSE}$]", fontsize=10)

# set plot legend
plt.legend(loc="upper right", numpoints=1, fancybox=True)

# add plot title
plt.title('Training Epochs $e_i$ vs. Prediction Error $L^{MSE}$', fontsize=10);

Great! The training error is decreasing nicely. We could train the network for a few more epochs until the error converges. However, let's stay with the 200 training epochs for now and proceed with evaluating our trained model.

## 7. Neural Network Model Evaluation

In this section, we will conduct a visual comparison of the predicted daily returns to the actual (true) daily returns. The comparison will include the daily returns of both the in-sample time period and the out-of-sample time period.

### 7.1. In-Sample Evaluation

Before starting our evaluation, let's load the best performing model or an already pre-trained model (as shown below). Remember that we stored a snapshot of the model after each training epoch in our local model directory. We will now load one of the (hopefully well-performing) saved snapshots.

In [ ]:
# init the pre-trained model architecture
lstm_model_pretrained = LSTMNet().to(device)

# restore pretrained model checkpoint

# define remote model path
github_model_path = 'https://raw.githubusercontent.com/HSG-AIML-Teaching/GSERM2024-Lab/main/lab_07/03_models/'

# define remote model name
lstm_model_name_pretrained = 'best_lstm_model_30000.pth'

# Read stored model from the remote location
lstm_bytes = urllib.request.urlopen(os.path.join(github_model_path, lstm_model_name_pretrained))

# Load tensor from io.BytesIO object
lstm_buffer = io.BytesIO(lstm_bytes.read())

# load trained models
lstm_model_pretrained.load_state_dict(torch.load(lstm_buffer, map_location='cpu'))

Let's check if the model was loaded successfully:

In [ ]:
# set model in evaluation mode
lstm_model_pretrained.eval()

Use the pre-trained model to determine the daily return predictions for the **in-sample** sequence population:

In [ ]:
# don't calculate gradients
with torch.no_grad():

    # predict sequence output
    train_predictions = lstm_model_pretrained(train_sequences_input.to(device))

    # collect prediction batch results
    train_predictions_list = train_predictions.cpu().detach().numpy()[:, -1].tolist()

    # collect target batch results
    train_targets_list = train_sequences_target.numpy()[:, -1].tolist()

Plot the pre-trained `LSTMNet` daily **in-sample** predictions against the target (*ground-truth*) daily returns:

In [ ]:
# set plot size
plt.rcParams['figure.figsize'] = [15, 5]

# prepare plot
fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(train_stock_sequence_data_date[:, -1], train_targets_list, color='C1', label='groundtruth (green)')
ax.plot(train_stock_sequence_data_date[:, -1], train_predictions_list, color='C0', label='predictions (blue)')

# set y-axis limits
ax.set_xlim(train_stock_sequence_data_date[:, -1].min(), train_stock_sequence_data_date[:, -1].max())

# set plot legend
plt.legend(loc="lower right", numpoints=1, fancybox=True)

# set plot title
plt.title('LSTM NN In-Sample Prediction vs. Ground-Truth Market Prices', fontsize=10)

# set axis labels
plt.xlabel('[time]', fontsize=8)
plt.ylabel('[market price]', fontsize=8)

# set axis ticks fontsize
plt.xticks(fontsize=8)
plt.yticks(fontsize=8);

### 7.2. Out-of-Sample Evaluation

Use the pre-trained model to determine the daily return predictions for the **out-of-sample** sequence population:

In [ ]:
# don't calculate gradients
with torch.no_grad():

    # predict sequence output
    valid_predictions = lstm_model_pretrained(valid_sequences_input.to(device))

    # collect prediction batch results
    valid_predictions_list = valid_predictions.cpu().detach().numpy()[:, -1].tolist()

    # collect target batch results
    valid_targets_list = valid_sequences_target.numpy()[:, -1].tolist()

Plot the pre-trained `LSTMNet` daily **out-of-sample** predictions against the target (*ground-truth*) daily returns:

In [ ]:
# set plot size
plt.rcParams['figure.figsize'] = [15, 5]

# prepare plot
fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(valid_stock_sequence_data_date[:, -1], valid_targets_list, color='C1', label='groundtruth (green)')
ax.plot(valid_stock_sequence_data_date[:, -1], valid_predictions_list, color='C0', label='predictions (blue)')

# set y-axis limits
ax.set_xlim(valid_stock_sequence_data_date[:, -1].min(), valid_stock_sequence_data_date[:, -1].max())

# set plot legend
plt.legend(loc="lower right", numpoints=1, fancybox=True)

# set plot title
plt.title('LSTM NN Out-of-Sample Prediction vs. Ground-Truth Market Prices', fontsize=10)

# set axis labels
plt.xlabel('[time]', fontsize=8)
plt.ylabel('[market price]', fontsize=8)

# set axis ticks fontsize
plt.xticks(fontsize=8)
plt.yticks(fontsize=8);

## 8. Neural Network Model Backtesting

In this section, we will backtest using the Python `bt` library. Python `bt` is a flexible backtesting framework that can be used to test quantitative trading strategies. In general, backtesting is the process of testing a strategy over a given data set (more details about the `bt` library can be found at: https://pmorissette.github.io/bt/.

To test the predictions derived from the LSTM model, we will view its predictions $\hat{r}_{i+1}$ as trade signals $\phi$. We will interpret any positive future return prediction $r_{t+1} > 0.0$ for a sequence $s^i$ as a "long" (buy) signal. Likewise, we will interpret any negative future return prediction $r_{t+1} < 0.0$ for a sequence $s$ as a "short" (sell) signal.

### 8.1. LSTM Trading Signal Preparation

Let's start by converting the out-of-sample model predictions into a trading signal, as described above. First, we convert the obtained predictions into a data frame that contains (1) the **date of the predicted returns** and (2) the **predicted returns $r_{t+1}$**:

In [ ]:
signal_data = pd.DataFrame(valid_predictions_list, columns=['PREDICTIONS'], index=valid_stock_sequence_data_date[:, -1])

Furthermore, let's verify the successful conversion by inspecting the top 10 rows of the created data frame:

In [ ]:
signal_data.head(10)

Now, let's derive a trading signal from the converted predictions. As previously described, we will generate the trading signal $\phi$ according to the following function:

<center>
$
\\
\phi(\hat{r}_{t+1})=
\begin{cases}
1.0, & for & \hat{r}_{t+1} > 0.0\\
-1.0, & for & \hat{r}_{t+1} < 0.0\\
\end{cases}
$
</center>

where $\hat{r}_{t+1}$ denotes a future return predicted by the model at time $t+1$.

In [ ]:
signal_data['SIGNAL'] = np.where(signal_data['PREDICTIONS'] > 0.0, 1.0, -1.0)

Let's inspect the top 10 rows of the generated trading signals:

In [ ]:
signal_data.head(10)

Let's now offset the prepared trading signal by a single day to $t-1$. This way, we rebalance our stock positions one day prior based on the closing price predicted by the `LSTMNet` model. As a result, we will be able to anticipate the model's future closing price prediction for a particular day $t$:

In [ ]:
signal_data = signal_data.set_index(signal_data['SIGNAL'].index - pd.DateOffset(1))

Let's inspect the top 10 rows of the prepared and offset trading signals:

In [ ]:
signal_data.head(10)

Visualize the predicted and prepared trading signals of the `LSTMNet` model:

In [ ]:
# set plot size
plt.rcParams['figure.figsize'] = [15, 5]

# prepare plot
fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(signal_data['SIGNAL'], lw=1.0, color='C3', label='LSTM trade signals')
    
# set axis ranges
ax.set_xlim([signal_data.index[0], signal_data.index[-1]])
ax.set_ylim([-1.1, 1.1])

# set axis labels
ax.set_xlabel('[time]', fontsize=10)
ax.set_ylabel('[lstm tade signal]', fontsize=10)

# rotate x-axis ticks
for tick in ax.get_xticklabels():
    tick.set_rotation(45)

# set plot title
ax.set_title('International Business Machines Corporation (IBM) - LSTM Trading Signals', fontsize=10);

Determine the number of trade signal changes (trades to be executed) within the out-of-sample timeframe from **03/2016** to **12/2017**, resulting in a total in-sample timeframe of approximately **21 months** (9 + 12):

In [ ]:
# determine number of signal changes
len(list(itertools.groupby(signal_data['SIGNAL'], lambda x: x > 0)))

On average, there are around **7** signal changes (trades) per month (148 signal changes / 21 months) within the out-of-sample time period.

### 8.2. Stock Market Data Preparation

Now, let's prepare the daily closing prices so they can be utilized in the backtest:

In [ ]:
stock_market_data = pd.DataFrame(stock_data['Close'])
stock_market_data = stock_market_data.rename(columns={'Close': 'PRICE'})
stock_market_data = stock_market_data.set_index(pd.to_datetime(stock_data.index))

Let's inspect the top 5 rows of the prepared closing prices:

In [ ]:
stock_market_data.head(5)

Sub-sample the prepared daily closing prices for the out-of-sample time period:

In [ ]:
stock_market_data = stock_market_data[stock_market_data.index >= signal_data.index[0]]
stock_market_data = stock_market_data[stock_market_data.index <= signal_data.index[-1]]

Let's inspect the top 5 rows of the prepared closing prices:

In [ ]:
stock_market_data.head(5)

Visualize the out-of-sample daily closing prices:

In [ ]:
# set plot size
plt.rcParams['figure.figsize'] = [15, 5]

# prepare plot
fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(stock_market_data['PRICE'], color='#9b59b6')

for tick in ax.get_xticklabels():
    tick.set_rotation(45)
    
# set axis labels
ax.set_xlabel('[time]', fontsize=10)
ax.set_ylabel('[equity %]', fontsize=10)

for tick in ax.get_xticklabels():
    tick.set_rotation(45)

# set axis labels and limits
ax.set_xlabel('[time]', fontsize=10)
ax.set_xlim([stock_market_data.index[0], stock_market_data.index[-1]])
ax.set_ylabel('[adj. closing price]', fontsize=10)

# set plot title
plt.title('International Business Machines Corporation (IBM) - Daily Historical Stock Closing Prices', fontsize=10);

Let's calculate the potentially gained return from applying a simple **"buy and hold"** strategy:

In [ ]:
np.abs(stock_market_data.iloc[0]['PRICE'] - stock_market_data.iloc[-1]['PRICE']) / stock_market_data.iloc[0]['PRICE']

With such a simple strategy, we would have been able to yield a total return of approximately **5.32%**.

### 8.3. Backtest Preparation

Now that we have trading signals as well as the market data, let's implement the LSTM-based trading strategy, which we will name `LSTMStrategy`:

In [ ]:
class LSTMStrategy(bt.Algo):
    
    def __init__(self, signals):
        
        # set class signals
        self.signals = signals
        
    def __call__(self, target):
        
        if target.now in self.signals.index[1:]:
            
            # get actual signal
            signal = self.signals[target.now]
            
            # set target weights according to signal
            target.temp['weights'] = dict(PRICE=signal)
            
        # return True since we want to move on to the next timestep
        return True

Let's instantiate our LSTM-based trading strategy:

In [ ]:
lstm_strategy = bt.Strategy('lstm', [bt.algos.SelectAll(), LSTMStrategy(signal_data['SIGNAL']), bt.algos.Rebalance()])

Initialize the backtest of our LSTM-based trading strategy using the strategy and prepared market data:

In [ ]:
backtest_lstm = bt.Backtest(strategy=lstm_strategy, data=stock_market_data, name='stock_lstm_backtest')

In addition, let's prepare a backtest of a "baseline" buy-and-hold trading strategy for comparison purposes. Our buy-and-hold strategy sends a "long" (+1.0) signal at each time step of the out-of-sample time frame:

In [ ]:
signal_data_base = signal_data.copy(deep=True) 
signal_data_base['SIGNAL'] = 1.0

Initialize the buy-and-hold ("base") strategy and the corresponding backtest:

In [ ]:
base_strategy = bt.Strategy('base', [bt.algos.SelectAll(), LSTMStrategy(signal_data_base['SIGNAL']), bt.algos.Rebalance()])
backtest_base = bt.Backtest(strategy=base_strategy, data=stock_market_data, name='stock_base_backtest')

### 8.4. Backtest Execution and Evaluation

Run the backtest for both trading strategies:

In [ ]:
backtest_results = bt.run(backtest_lstm, backtest_base)

Inspect the individual backtest results and performance measures:

In [ ]:
backtest_results.display()

Collect detailed backtest performance per time step of the LSTM trading strategy:

In [ ]:
backtest_lstm_details = backtest_lstm.strategy.prices.to_frame(name='Rel. EQUITY')
backtest_lstm_details['Abs. EQUITY'] = backtest_lstm.strategy.values # equity per timestep
backtest_lstm_details['CASH'] = backtest_lstm.strategy.cash # cash per timestep
backtest_lstm_details['POSITIONS'] = backtest_lstm.strategy.positions # positions per timestep
backtest_lstm_details['FEES'] = backtest_lstm.strategy.fees # trading fees per timestep

Inspect the LSTM trading strategy backtest details:

In [ ]:
backtest_lstm_details.head(10)

Visualize the monthly returns obtained by the LSTM-based trading strategy:

In [ ]:
# set plot size
plt.rcParams['figure.figsize'] = [15, 5]

# prepare plot
fig = plt.figure()
ax = fig.add_subplot(111)

# plot heatmap of monthly returns generated by the strategy
ax = sns.heatmap(backtest_lstm.stats.return_table, annot=True, cbar=True, vmin=-0.5, vmax=0.5)

# set axis labels
ax.set_xlabel('[month]', fontsize=10)
ax.set_ylabel('[year]', fontsize=10)

# set plot title
ax.set_title('International Business Machines Corporation (IBM) - Monthly Returns LSTM Strategy', fontsize=10);

Collect detailed backtest performance per time step of the "buy-and-hold" trading strategy:

In [ ]:
backtest_base_details = backtest_base.strategy.prices.to_frame(name='Rel. EQUITY')
backtest_base_details['Abs. EQUITY'] = backtest_base.strategy.values # equity per timestep
backtest_base_details['CASH'] = backtest_base.strategy.cash # cash per timestep
backtest_base_details['POSITIONS'] = backtest_base.strategy.positions # positions per timestep
backtest_base_details['FEES'] = backtest_base.strategy.fees # trading fees per timestep

Inspect the "buy-and-hold" trading strategy backtest details:

In [ ]:
backtest_base_details.head(10)

Visualize the monthly returns obtained by the "buy-and-hold" trading strategy:

In [ ]:
# set plot size
plt.rcParams['figure.figsize'] = [15, 5]

# prepare plot
fig = plt.figure()
ax = fig.add_subplot(111)

# plot heatmap of monthly returns generated by the strategy
ax = sns.heatmap(backtest_base.stats.return_table, annot=True, cbar=True, vmin=-0.5, vmax=0.5)

# set axis labels
ax.set_xlabel('[month]', fontsize=10)
ax.set_ylabel('[year]', fontsize=10)

# set plot title
ax.set_title('International Business Machines Corporation (IBM) - Monthly Returns \'buy-and-hold\' Strategy', fontsize=10);

Visualize the equity progression of both strategies over time:

In [ ]:
# set plot size
plt.rcParams['figure.figsize'] = [15, 5]

# prepare plot
fig = plt.figure()
ax = fig.add_subplot(111)

# plot relative equity per trading strategy
ax.plot(backtest_lstm_details['Rel. EQUITY'], color='green',lw=1.0, label='lstm strategy (green)')
ax.plot(backtest_base_details['Rel. EQUITY'], color='gray',lw=1.0, label='base strategy (gray)')

for tick in ax.get_xticklabels():
    tick.set_rotation(45)
    
# set axis labels
ax.set_xlabel('[time]', fontsize=10)
ax.set_xlim(valid_stock_sequence_data_date[:, -1].min(), valid_stock_sequence_data_date[:, -1].max())
ax.set_ylabel('[equity %]', fontsize=10)

# set plot legend
plt.legend(loc="upper right", numpoints=1, fancybox=True)

# set plot title
plt.title('International Business Machines Corporation (IBM) - Backtest % Equity Progression', fontsize=10);

## 9. Lab Summary:

In this lab, you successfully accomplished the following key learnings:

> 1. **Understanding Long Short-Term Memory (LSTM) Neural Network Design:** Mastered the fundamental concepts and architectural design of LSTM neural networks, enhancing your comprehension of deep learning models tailored for sequential data analysis in financial time series.
> 2. **Model Implementation and Training:** Developed practical skills in implementing and training an LSTM model using PyTorch, applying it to historical financial data to predict future stock prices.
> 3. **Evaluating Model Performance:** Gained expertise in evaluating the performance of LSTM models through metrics such as loss and accuracy, effectively utilizing these metrics to assess the model's predictive capabilities.
> 4. **Visualization and Interpretation of Results:** Learned to visualize and interpret the model's predictions, providing deeper insights into the temporal patterns of the data and facilitating the contextualization of prediction results.

This lab provided insights into designing, implementing, training, and evaluating LSTMs for sequential data prediction. It equipped you with tools and techniques for effective LSTM model building, evaluation, and application.